In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [972 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [28.5 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,060 kB]
Get:13 http://archive.ubuntu.com/ubuntu f

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [4]:
tv_df = df.filter(df.total_votes >= 20)
tv_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36886257|R1T4FVSABO0IDP|B00V03D9KY|     966609438|Lukas Lk-7950 WD ...|      Automotive|          5|            8|         31|   N|                Y|      Great Value!!!|LK-7950 is an out...| 2015-08-31|
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|    

In [5]:
tv_fifty = tv_df.filter((tv_df.helpful_votes / tv_df.total_votes) >= 0.5)
tv_fifty.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|           21|         21|   N|                Y|Awesome bumper. I...|Awesome bumper. I...| 2015-08-31|
|         US|   11359139|R2H6KB9RHS17GA|B00VNBDWPK|     790984844|OPT7 Fluxbeam LED...|      Automotive|          3|    

In [6]:
paid_df = tv_fifty.filter(tv_fifty.vine == 'Y')
paid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35560012| REKN4PLM5174P|B00P9OF04I|      44916596|Meguiar's Heavy D...|      Automotive|          4|          219|        225|   Y|                N|Head 2 Head, It's...|Having come into ...| 2015-07-08|
|         US|   52693410| RQOG8IIJX7CKB|B00P9OF04I|      44916596|Meguiar's Heavy D...|      Automotive|          4|    

In [7]:
unpaid_df = tv_fifty.filter(tv_fifty.vine == 'N')
unpaid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|           21|         21|   N|                Y|Awesome bumper. I...|Awesome bumper. I...| 2015-08-31|
|         US|   11359139|R2H6KB9RHS17GA|B00VNBDWPK|     790984844|OPT7 Fluxbeam LED...|      Automotive|          3|    

In [8]:
# Paid total number of reviews
tnr = paid_df.count()
tnr

82

In [9]:
# Paid number of 5-star reviews
n5r = paid_df.filter(paid_df.star_rating == 5).count()
n5r

33

In [11]:
# Paid percentage of 5-star reviews
p5r = (n5r / tnr) * 100
p5r

40.243902439024396

In [12]:
# Unpaid total number of reviews
tur = unpaid_df.count()
tur

24742

In [13]:
# Unpaid number of 5-star reviews
u5r = unpaid_df.filter(unpaid_df.star_rating == 5).count()
u5r

12807

In [14]:
# Unpaid percentage of 5-star reviews
pur = (u5r / tur) * 100
pur

51.76218575701237